In [63]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu' # use cuda(gpu) if not available use cpu
#print (device)
block_size = 8 # how many intigers can the model see at once
batch_size = 4 # how many blocks can the model see at once
max_iters = 1000
eval_iters = 250
learning_rate = 3e-4
#dropout = 0.2 # this helps the model to generalize, to not depend on only some neurons by randomly turning some off (0.2 = 20%), only active during training mode 

In [64]:
with open ('wizard_of_oz.txt', 'r', encoding='utf-8') as f: # use wizard_of_oz.txt as a text and encode it with utf-8
    text = f.read() # let the script read the text
chars = sorted(set(text)) # sort all used characters in text
#print(chars)
vocab_size = len(chars) # sets the vocab_size to the number of characters in chars

In [65]:
string_to_int = { ch:i for i,ch in enumerate(chars) } # make string of characters into intigers (full numbers)
int_to_string = { i:ch for i,ch in enumerate(chars) } # make intigers into string of characters
encode = lambda s: [string_to_int[c] for c in s] # encode the characters into the intigers
decode = lambda l: ''.join([int_to_string[i] for i in l]) # decode the intigers into characters

data = torch.tensor(encode(text), dtype=torch.long) # turns encoded text into tensors
#print(data[:100])

In [66]:
n = int(0.8*len(data)) # take only 80% of the data
train_data = data[:n] # use the 80% for training
val_data = data [n:] # use the remaining 20% for validation

def get_batch(split): # defines a function called get_batch that takes a single argument split, which tells us whether to use the training or validation data
    data = train_data if split == 'train' else val_data # This selects which dataset to use: If split is 'train', use train_data.Otherwise use val_data.
    ix = torch.randint(len(data) - block_size, (batch_size,)) # Each index i represents the start of a chunk of length block_size. We subtract block_size to ensure we don’t go out of bounds.
    #print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix]) # For each index i in ix, it slices a chunk from data of length block_size. Then stacks them into a tensor of shape (batch_size, block_size)
    y = torch.stack([data[i+1:i+block_size+1]for i in ix]) # This creates the target batch y, which is the same as x but shifted by one token to the right.The model will try to predict each next token in y given the corresponding token in x
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
#print('inputs:')
#print(x.shape)
#print(x)
#print('targets:')
#print(y)


In [67]:
# makes sure that torch doesnt use gradients at all so that will reduce computation, memoryusage just better for performance
@torch.no_grad() # build in pytorch decorator that disbles any gradient calculations
def estimate_loss():
    out = {}
    model.eval() # switches to evaluation mode, disables dropout
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # switches to training mode, enables dropout
    return out

In [68]:
class BiagramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # This creates a table that turns each word into a prediction for the next word.
        # It's like saying: "If I see word X, what word should come next?"
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        # Look up each input word in the embedding table.
        # The result is a guess (logits) for the next word at each position.
        logits = self.token_embedding_table(index)  # shape: (B, T, vocab_size)

        if targets is None:
            # If we're just using the model (not training), no need for a loss.
            loss = None
        else:
            # We're training the model, so calculate how wrong the guesses were.
            # First, flatten the logits and targets to make them easier to compare.
            B, T, C = logits.shape 
            logits = logits.view(B*T, C)         # make logits 2D: (all tokens, vocab_size)
            targets = targets.view(B*T)          # flatten targets too: (all tokens)
            # Now calculate how far off the model’s guesses were.
            loss = F.cross_entropy(logits, targets)

        # Return both the guesses and the loss (loss is None during generation)
        return logits, loss

    def generate(self, index, max_new_tokens):
        # This function makes the model keep adding words to the input.
        # Example: start with "The", it might add "cat", then "sat", etc.

        for _ in range(max_new_tokens):
            # Get the model's guess for the next word.
            logits, _ = self.forward(index)  # shape: (B, T, vocab_size)

            # Focus only on the last word's prediction.
            logits = logits[:, -1, :]  # shape: (B, vocab_size)

            # Turn the logits into probabilities (so we can pick randomly but fairly)
            probs = F.softmax(logits, dim=-1)  # shape: (B, vocab_size)

            # Randomly pick the next word using those probabilities.
            index_next = torch.multinomial(probs, num_samples=1)  # shape: (B, 1)

            # Add the new word to the end of the sequence.
            index = torch.cat((index, index_next), dim=1)  # shape: (B, T+1)

        # Return the full sequence, including the new words we added.
        return index


model = BiagramLanguageModel(vocab_size) # make the model and tell it how many tokens it can know
m = model.to(device) # put the model on GPU if available otherwise CPU

context = torch.zeros((1,1), dtype=torch.long, device=device) # starts with a token of 0, .long cus the tokens must be integers, uses device to generate
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist()) # generate 500 new tokens using the model starting from context .generate returns a tensor of token ids, .tolist converts the ids into clear integers, decode makes the integers into characters 
#print(generated_chars)

In [84]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate) # creates AdamW optimizer instance to update the models parameters

for iter in range(max_iters): # starts a training loop that runs for the duration of max_iters (1000) each iteration performs an update step
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    
    xb, yb = get_batch('train') # fetches a batch of input data (xb) and target labels (yb) from training dataset (wiz_of_oz)
    logits, loss = model.forward(xb, yb)  # passes the input and target batches thrugh the model, returns logits tyhe raw output and the loss scalar comparing the predictions and targets
    optimizer.zero_grad(set_to_none=True) # clears previos gradients from the model parameters
    loss.backward() # computes gradients of the loss with respect to all model parameters
    optimizer.step() # updates model parameters using the computed gradients and the optimizer algorithm
print(loss.item()) # .item converts the tensor into a readable number 


step: 0, train loss: 2.853, val loss: 2.866
step: 250, train loss: 2.842, val loss: 2.861
step: 500, train loss: 2.806, val loss: 2.848
step: 750, train loss: 2.806, val loss: 2.815
2.6351840496063232


In [51]:
context = torch.zeros((1,1), dtype=torch.long, device=device) # creates a tensor filled with zeros of the shape (1, 1) 1 batch 1 token 
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist()) # generates 500 tokens from a starting point context .tolist makes it into ids and decode into characters
print(generated_chars)


r s bu gre
d "I   u wat WARI wigaritherbupan, ond bl d wifry wace
ther " the Jin, if mon! toun ththebe withe u moth as the ld, tlebekere n * he bey tifr  wime he tr Scr  WImait
I d 'Oz,"AGin tat jors thathoashelews Doug ove Dope s thy.
"Nofintodome cas lligho int Ozat les cofthe, g ase d the t BRa, ply w s, nd pllinthisaid; amabey her pe ctt, y thers knsco,'vithout a s basthe  whind f s; l.

blely tintht bof avend Allithen
enthe qushide ts n tlafet
" oren
"Mainted ued d ain Oftl st "
sas

whirf 
